# First we will begin with a basic review of numpy

In [48]:
import numpy as np

In [2]:
a = np.arange(5)

In [4]:
b = np.arange(10,15)

In [5]:
a.dot(b)

130

In [7]:
np.sum(a*b)

130

In [8]:
a*b

array([ 0, 11, 24, 39, 56])

In [9]:
row_vector = np.array([[1, 3, 2]])

In [10]:
col_vector = np.array([[2], [0], [1]])

In [11]:
row_vector.dot(col_vector)

array([[4]])

In [12]:
col_vector.dot(row_vector)

array([[2, 6, 4],
       [0, 0, 0],
       [1, 3, 2]])

In [14]:
col_vector.ravel(),row_vector.ravel()

(array([2, 0, 1]), array([1, 3, 2]))

In [15]:
np.outer(col_vector.ravel(),row_vector.ravel())

array([[2, 6, 4],
       [0, 0, 0],
       [1, 3, 2]])

# Now onto the real stuff
### Comparing word embeddings to fill in the blank

In [65]:
import sys,random,math
from collections import Counter
from nltk import word_tokenize

In [66]:
#Add a seed for randomizing choice which will be explained later
np.random.seed(1)

In [88]:
#This is a list of movie reviews of Imdb file will be uploaded to github later.Example from Andrew Trask's book Grokking
#Deep Learning

f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

In [70]:
#tokens = word_tokenize(raw_reviews)

In [89]:
#tokens
#split the review array into small arrays of words of each sentence

tokens = list(map(lambda x:(x.split(" ")),raw_reviews))

In [90]:
wordcnt = Counter()

In [91]:
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1

In [92]:
# String operations to remove punctuations
import string

In [75]:
## We use the popular stopwords defined in the nltk corpus. 
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


In [76]:
stop_words = set(stopwords.words('english'))

In [123]:
#tokens = [w for w in tokens if w not in stop_words]

In [79]:
for word in tokens:
    wordcnt[word] -= 1

In [93]:
vocab = list(set(map(lambda x:x[0],wordcnt.most_common())))

In [94]:
len(vocab)

74075

### Create word to index and reverse dictionary

In [95]:
word2index = {}
index2word = {}
for i,word in enumerate(vocab):
    word2index[word]=i
    index2word[i]=word

In [96]:
index2word[6085],word2index['scopes']

('instigation', 9357)

In [97]:
concatenated = list()
input_dataset = list()

In [98]:
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            ""
    input_dataset.append(sent_indices)
    

In [102]:
#concatenated is a list of all the indices of the vocab
#input_dataset is a list of sentences with indices of words replacing the actual words
concatenated = np.array(concatenated)
random.shuffle(input_dataset)

In [115]:
alpha, iterations = (0.05, 2)
hidden_size,window,negative = (50,2,5)
weights_0_1 = (np.random.rand(len(vocab),hidden_size) - 0.5) * 0.2
weights_1_2 = np.random.rand(len(vocab),hidden_size)*0
layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1
def similar(target='beautiful'):
    target_index = word2index[target]

    scores = Counter()
    for word,index in word2index.items():
        
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
    return scores.most_common(10)


In [116]:
weights_0_1

array([[ 0.04962836,  0.01988407, -0.02924894, ...,  0.02235409,
        -0.07866708, -0.08212026],
       [-0.02437445,  0.06014989, -0.04563997, ..., -0.03621135,
        -0.09106698, -0.0571341 ],
       [-0.05086565,  0.01672281,  0.02537102, ...,  0.04611479,
        -0.06692812, -0.09473534],
       ..., 
       [ 0.04327146,  0.06471183,  0.06579626, ..., -0.08170339,
         0.03575512, -0.09422431],
       [ 0.09846515,  0.05407922, -0.07341868, ...,  0.02439074,
        -0.01956672, -0.02716743],
       [ 0.09529182,  0.00331547, -0.05528874, ..., -0.08805801,
        -0.07556738, -0.00967705]])

In [117]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [118]:
import sys,random,math

In [119]:
for rev_i,review in enumerate(input_dataset * 10):
    for target_i in range(len(review)):
        
        target_samples = [review[target_i]]+list(concatenated\
    [(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])
        left_context = review[max(0,target_i-window):target_i]
        right_context = review[target_i+1:min(len(review),target_i+window)]
        layer_1 = np.mean(weights_0_1[left_context+right_context],axis=0)
        layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
        layer_2_delta = layer_2 - layer_2_target
        layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])
        weights_0_1[left_context+right_context] -= layer_1_delta * alpha
        weights_1_2[target_samples] -= np.outer(layer_2_delta,layer_1)*alpha
    if(rev_i % 250 == 0):
        sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)) + "   " + str(similar('terrible')))
        sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)))
        

Progress:4.995

In [120]:
print(similar('terrible'))

[('terrible', -0.0), ('horrible', -2.4754866203702597), ('lame', -3.4387048575178283), ('dreadful', -3.5539998565731437), ('ridiculous', -3.6751871768620124), ('pathetic', -3.6799033008784447), ('brilliant', -3.8682180919265168), ('bad', -3.8752407833336346), ('stupid', -3.9329262050358285), ('dull', -4.038289791874596)]


In [139]:
weights_0_1

array([[-0.02309237, -0.33961461, -0.61860407, ..., -0.44784579,
        -0.8079342 ,  0.02609339],
       [-0.3543827 , -0.45861281,  0.04627791, ...,  0.098861  ,
         0.06744006,  0.609333  ],
       [-0.08620186,  0.24950204,  0.01597825, ...,  0.24110192,
         0.07037026,  0.76157882],
       ..., 
       [-0.16956576, -0.35832148,  0.29649288, ..., -0.25356524,
        -0.18207844,  0.29944431],
       [-0.70936176,  0.12017149,  0.41595295, ...,  0.52804974,
         0.20563586,  0.32338842],
       [ 0.35931363,  0.53415009,  0.0473822 , ..., -0.06943643,
        -0.08977681, -0.35579795]])

In [121]:
weights_0_1

array([[-0.02309237, -0.33961461, -0.61860407, ..., -0.44784579,
        -0.8079342 ,  0.02609339],
       [-0.3543827 , -0.45861281,  0.04627791, ...,  0.098861  ,
         0.06744006,  0.609333  ],
       [-0.08620186,  0.24950204,  0.01597825, ...,  0.24110192,
         0.07037026,  0.76157882],
       ..., 
       [-0.16956576, -0.35832148,  0.29649288, ..., -0.25356524,
        -0.18207844,  0.29944431],
       [-0.70936176,  0.12017149,  0.41595295, ...,  0.52804974,
         0.20563586,  0.32338842],
       [ 0.35931363,  0.53415009,  0.0473822 , ..., -0.06943643,
        -0.08977681, -0.35579795]])

In [125]:
[review[target_i]]+list(concatenated\
    [(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])

[33061, 16378, 0, 57550, 65722, 0]

In [127]:
similar('beautiful')

[('beautiful', -0.0),
 ('gorgeous', -3.307808934165823),
 ('lovely', -3.707893770910298),
 ('creepy', -3.886914105627391),
 ('mountainside', -3.934657007488936),
 ('crotchety', -3.9376593450795383),
 ('charming', -4.035929591981383),
 ('colourful', -4.064878067516477),
 ('luxuriant', -4.106692654897076),
 ('girlie', -4.115702426529915)]

## Adding word analogy caluclation
### To add word analogy calculation we calculate the sum of the vectors and balance the equation

In [130]:
np.sum(weights_0_1 * weights_0_1,axis=1).reshape(74075,1)

array([[ 16.98399894],
       [  4.61947552],
       [ 13.9349861 ],
       ..., 
       [  5.03685611],
       [ 17.87903092],
       [  4.74135481]])

In [136]:
normed_weight = (weights_0_1*(np.sum(weights_0_1 * weights_0_1,axis=1).reshape(74075,1)))

In [137]:
normed_weight[44310]

array([ -16.37061507,   34.99174838,   -4.20912844,   56.76490684,
         52.03343404,  -11.73584499,  -68.11704798,  -31.36502876,
        -23.91079598,  -25.98866284,   65.47682637,  -18.18969689,
        -30.3837959 ,  -47.59354102,   99.42739835,   -5.72706995,
       -112.03355731,    6.93024198,   49.33171613,  -10.08592312,
         24.835001  ,  -33.20260794, -109.35777844,    5.91589421,
         33.68829907,  -38.83598405,   -6.18332416,   -9.41422427,
         45.36255933,   -2.10007775,  -52.15353617,   -8.37228563,
        -14.90395947,   22.00199817,  -34.81305928,   90.18007781,
        -30.36409035,  -58.0360945 ,   21.91633809,  -13.77610488,
          1.41813938,   53.07162629,  -29.82252822,   42.86978666,
        114.51527076,  -25.28018723,   36.05084705,   -0.38610472,
         -3.51687398,   12.05135696])

In [146]:
def analogy_similar(analogy1=["king","man"],analogy2=["queen"]):
    norms = np.sum(weights_0_1 * weights_0_1,axis=1)
    norms.resize(norms.shape[0],1)
    normed_weights = weights_0_1 * norms
    query_vect = np.zeros(len(weights_0_1[0]))
    for word in analogy1:
        query_vect +=  normed_weights[word2index[word]]
    for word in analogy2:
        query_vect -= normed_weights[word2index[word]]
    
    scores = Counter()
    for word,index in word2index.items():
        raw_diff = weights_0_1[index] - query_vect
        sq_error = raw_diff ** 2
        scores[word] = -math.sqrt(sum(sq_error))
    return scores.most_common(10)[1:]
        
    

In [147]:
analogy_similar()  ##king + man - woman = doctor???

[('\n', -429.6068169910971),
 ('man', -430.06559087780136),
 ('king', -430.1363616449812),
 ('guy', -430.3113933681435),
 ('viewer', -430.46956154540084),
 ('reviewer', -430.69623711532455),
 ('doctor', -430.86925845641196),
 ('killer', -430.94380406849064),
 ('woman', -430.98176061560775)]

In [149]:
analogy_similar(["elizabeth","he"],["she"]) ##not bad

[('eli', -323.736247590786),
 ('jonathan', -324.0112338682167),
 ('leonard', -324.1114241654765),
 ('karl', -324.2021728697042),
 ('edgar', -324.27783310890186),
 ('stephen', -324.3097192410623),
 ('le', -324.3659046660408),
 ('jon', -324.36947828113176),
 ('juliette', -324.4124575442978)]